In [6]:
import numpy as np
# from GAIndividual import GAIndividual
import random
import copy
import matplotlib.pyplot as plt
import main_func

class GeneticAlgorithm(object):
    '''
    The class for genetic algorithm
    '''
    def __init__(self, sizepop, vardim, bound, MAXGEN, params):
        '''
        sizepop: population sizepop 種群數量 60
        vardim: dimension of variables 變量維度 3
        bound: boundaries of variables 變量邊界 -10 10
        MAXGEN: termination condition  迭代次數  1000
        param: 交叉率, 變異率, alpha = [0.9, 0.1, 0.5]
        '''
        self.sizepop = sizepop
        self.MAXGEN = MAXGEN
        self.vardim = vardim
        self.bound = bound
        self.population = []
        self.fitness = np.zeros((self.sizepop, 1))
        self.trace = np.zeros((self.MAXGEN, 2))
        self.params = params

    def initialize(self):
        '''
        initialize the population
        '''
        for i in range(0, self.sizepop):
            ind = GAIndividual(self.vardim, self.bound)
            ind.generate()
            self.population.append(ind)

    def evaluate(self):
        '''
        evaluation of the population fitnesses
        '''
        for i in range(0, self.sizepop):
            self.population[i].calculateFitness()

            self.fitness[i] = self.population[i].fitness

    def solve(self):
        '''
        evolution process of genetic algorithm
        '''
        print('solve')
        self.t = 0
        self.initialize()
        self.evaluate()
        best = np.max(self.fitness)
        bestIndex = np.argmax(self.fitness)
        self.best = copy.deepcopy(self.population[bestIndex])
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
        self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
        print("Generation %d: optimal function value is: %f; average function value is %f" % (
            self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
        while (self.t < self.MAXGEN - 1):
            self.t += 1
            self.selectionOperation()
            self.crossoverOperation()
            self.mutationOperation()
            self.evaluate()
            best = np.max(self.fitness)
            bestIndex = np.argmax(self.fitness)
            if best > self.best.fitness:
                self.best = copy.deepcopy(self.population[bestIndex])
            self.avefitness = np.mean(self.fitness)
#             self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
#             self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
            self.trace[self.t, 0] = self.best.fitness
            self.trace[self.t, 1] = self.avefitness
            print("Generation %d: optimal function value is: %f; average function value is %f" % (
                self.t, self.trace[self.t, 0], self.trace[self.t, 1]))

        print("Optimal function value is: %f; " %
              self.trace[self.t, 0])
        print ("Optimal solution is:")
        print (self.best.chrom)
        self.printResult()
        return self.best.chrom
    def selectionOperation(self):
        '''
        selection operation for Genetic Algorithm
        '''
        newpop = []
        totalFitness = np.sum(self.fitness)
        accuFitness = np.zeros((self.sizepop, 1))

        sum1 = 0.
        for i in range(0, self.sizepop):
            accuFitness[i] = sum1 + self.fitness[i] / totalFitness
            sum1 = accuFitness[i]

        for i in range(0, self.sizepop):
            r = random.random()
            idx = 0
            for j in range(0, self.sizepop - 1):
                if j == 0 and r < accuFitness[j]:
                    idx = 0
                    break
                elif r >= accuFitness[j] and r < accuFitness[j + 1]:
                    idx = j + 1
                    break
            newpop.append(self.population[idx])
        self.population = newpop

    def crossoverOperation(self):
        '''
        crossover operation for genetic algorithm
        '''
        newpop = []
        for i in range(0, self.sizepop, 2):
            idx1 = random.randint(0, self.sizepop - 1)
            idx2 = random.randint(0, self.sizepop - 1)
            while idx2 == idx1:
                idx2 = random.randint(0, self.sizepop - 1)
            newpop.append(copy.deepcopy(self.population[idx1]))
            newpop.append(copy.deepcopy(self.population[idx2]))
            r = random.random()
            if r < self.params[0]:
                crossPos = random.randint(1, self.vardim - 1)
                for j in range(crossPos, self.vardim):
                    newpop[i].chrom[j] = newpop[i].chrom[
                        j] * self.params[2] + (1 - self.params[2]) * newpop[i + 1].chrom[j]
                    newpop[i + 1].chrom[j] = newpop[i + 1].chrom[j] * self.params[2] + \
                        (1 - self.params[2]) * newpop[i].chrom[j]
        self.population = newpop

    def mutationOperation(self):
        '''
        mutation operation for genetic algorithm
        '''
        newpop = []
        for i in range(0, self.sizepop):
            newpop.append(copy.deepcopy(self.population[i]))
            r = random.random()
            if r < self.params[1]:
                mutatePos = random.randint(0, self.vardim - 1)
                theta = random.random()
                if theta > 0.5:
                    newpop[i].chrom[mutatePos] = newpop[i].chrom[
                        mutatePos] - (newpop[i].chrom[mutatePos] - self.bound[0, mutatePos]) * (1 - random.random() ** (1 - self.t / self.MAXGEN))
                else:
                    newpop[i].chrom[mutatePos] = newpop[i].chrom[
                        mutatePos] + (self.bound[1, mutatePos] - newpop[i].chrom[mutatePos]) * (1 - random.random() ** (1 - self.t / self.MAXGEN))
        self.population = newpop

    def printResult(self):
        '''
        plot the result of the genetic algorithm
        '''
        x = np.arange(0, self.MAXGEN)
        y1 = self.trace[:, 0]
        y2 = self.trace[:, 1]
        plt.plot(x, y1, 'r', label='optimal value')
        plt.plot(x, y2, 'g', label='average value')
        plt.xlabel("Iteration")
        plt.ylabel("function value")
        plt.title("Genetic algorithm for function optimization")
        plt.legend()
        plt.show()



In [ ]:
%%time
if __name__ == "__main__":
    global df
    camp_id = 23843355587140564
    df = ObjectiveFunc.account_status(23843355587140564)
    bound = np.tile([[0.5], [1]], 3)
    ga = GeneticAlgorithm(1000, 3, bound, 10, [0.9, 0.5, 0.5])
    optimal = ga.solve()
    score = ObjectiveFunc.fitnessfunc(optimal, df)
    score_columns=['weight_kpi', 'weight_spend', 'weight_bid']
    df_score = pd.DataFrame(data=[optimal], columns=['weight_kpi', 'weight_spend', 'weight_bid'], index=[0])
    
    df_final = pd.DataFrame({'campaign_id':camp_id, 'score':score}, columns=['campaign_id', 'score'], index=[0])
    df_final = pd.concat( [df_score, df_final], axis=1, sort=True, ignore_index=False)
    

In [7]:
import numpy as np
# import ObjFunction


class GAIndividual(object):
    '''
    individual of genetic algorithm
    '''

    def __init__(self,  vardim, bound):
        '''
        vardim: dimension of variables
        bound: boundaries of variables
        '''
        self.vardim = vardim
        self.bound = bound
        self.fitness = 0.

    def generate(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        len = self.vardim
        rnd = np.random.random(size=len)
        self.chrom = np.zeros(len)
        for i in range(0, len):
            self.chrom[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd[i]
    def calculateFitness(self):
        '''
        calculate the fitness of the chromsome
        '''
        self.fitness = ObjectiveFunc.fitnessfunc( self.chrom, df )


In [8]:
from main_func import Campaigns
from main_func import Accounts
import pandas as pd
import mysql_adactivity_save
class ObjectiveFunc(object):
    '''
    objective function of genetic algorithm
    '''
    def fitnessfunc( optimal_weight, df):
        w_kpi   = optimal_weight[0]
        w_spend = optimal_weight[1]
        w_bid   = optimal_weight[2]
#         w_width = optimal_weight[3]

        m_kpi   = df['campaign_charge'] / df['charge_per_day']
        m_spend = -( df['budget_per_day'] - df['spend'] ) / df['budget_per_day']
        m_bid   = ( df['campaign_bid'] - df['campaign_cpc'] ) / df['campaign_bid']
#         m_width = df['impressions'] / df['budget_per_day']
#         m_ctr   = ctr/target_ctr        

        status  = np.array( [m_kpi, m_spend, m_bid] )
        r = np.dot( optimal_weight, status )
        return r
    
    def adset_fitness(optimal_weight, df):
        m_kpi   = df['charge'] / df['charge_per_day']
        m_spend = -( df['budget_per_day'] - df['spend'] ) / df['budget_per_day']
        m_bid   = ( df['bid_amount'] - df['charge_cpc'] ) / df['bid_amount']
    #     m_width = df['impressions'] / df['daily_budget']
        status  = np.array( [m_kpi, m_spend, m_bid] )
        optimal_weight = np.array([df_score['weight_kpi'].iloc[0], df_score['weight_spend'].iloc[0], df_score['weight_bid'].iloc[0]])
        r = np.dot( optimal_weight, status )
        return r    
    
    def account_status( campaign_id ):
        mydb = mysql_adactivity_save.connectDB( "ad_activity" )
        df_camp = pd.read_sql("SELECT * FROM campaign_target WHERE campaign_id=%s" %(campaign_id), con=mydb)
        df_camp['charge_per_day'] = df_camp['target']/df_camp['campaign_days']
        df_camp['campaign_bid'] = df_camp['spend_cap']/df_camp['target']

        acc_id = Campaigns(campaign_id).get_account_id()
        insights = Accounts( "act_"+ acc_id ).get_account_insights()

        for insight in insights:
            spend = int( insight.get("spend") )
            account_cpc = float( insight.get("cpc") )
            account_charge = int( insight.get("clicks") )
            impressions = int( insight.get("impressions") )
        df=pd.DataFrame({'campaign_id':[campaign_id], 'campaign_cpc':[account_cpc], 'campaign_charge':[account_charge],
                         'impressions':[impressions], 'campaign_bid':[df_camp['campaign_bid'].iloc[0]], 'spend':[spend],
                         'daily_budget':[df_camp['budget_per_day'].iloc[0]], 'charge_per_day':[df_camp['charge_per_day'].iloc[0]], 'budget_per_day':[df_camp['budget_per_day'].iloc[0]]})

        df = df.convert_objects(convert_numeric=True)
        return df

    def campaign_status( campaign_id ):
        mydb = mysql_adactivity_save.connectDB( "ad_activity" )
        df_camp = pd.read_sql("SELECT * FROM campaign_target WHERE campaign_id=%s" %(campaign_id), con=mydb)
        df_camp['charge_per_day'] = df_camp['target']/df_camp['campaign_days']
        df_camp['campaign_bid'] = df_camp['spend_cap']/df_camp['target']

#         df_temp = pd.concat( df_camp[['campaign_id', 'charge_per_day', 'budget_per_day', 'daily_budget','']])
        insights = Campaigns(campaign_id).get_campaign_insights()
        for insight in insights:
            spend = int( insight.get("spend") )
            campaign_cpc = float( insight.get("cpc") )
            campaign_charge = int( insight.get("clicks") )
            impressions = int( insight.get("impressions") )
        df=pd.DataFrame({'campaign_id':[campaign_id], 'campaign_cpc':[campaign_cpc], 'campaign_charge':[campaign_charge],
                         'impressions':[impressions], 'campaign_bid':[df_camp['campaign_bid'].iloc[0]], 'spend':[spend],
                         'daily_budget':[df_camp['budget_per_day'].iloc[0]], 'charge_per_day':[df_camp['charge_per_day'].iloc[0]], 'budget_per_day':[df_camp['budget_per_day'].iloc[0]]})
        df = df.convert_objects(convert_numeric=True)
        return df
    def adset_status( ad_id ):
        mydb = mysql_adactivity_save.connectDB( "ad_activity" )

        df=pd.DataFrame({'adset_id':[],'charge':[], 'impressions':[], 'bid_amount':[]})
        
        df_ad = pd.read_sql("SELECT * FROM ad_insights WHERE ad_id=%s ORDER BY request_time DESC LIMIT 1" %(ad_id), con=mydb)
        df_adset = pd.read_sql("SELECT * FROM adset_insights WHERE adset_id=%s ORDER BY request_time DESC LIMIT 1" %(df_ad['adset_id'].iloc[0]), con=mydb)
        df_camp = pd.read_sql("SELECT * FROM campaign_target WHERE campaign_id=%s" %(df_ad['campaign_id'].iloc[0]), con=mydb)
        df_camp['charge_per_day'] = df_camp['target']/df_camp['campaign_days']
        df_temp = pd.merge( df_ad[['campaign_id', 'adset_id', 'charge', 'charge_cpc', 'impressions']],
                              df_adset[['adset_id','spend','bid_amount','daily_budget']],
                              on=['adset_id'] )
        df_status = pd.merge( df_temp,
                              df_camp[['campaign_id', 'charge_per_day', 'budget_per_day' ]],
                              on=['campaign_id'] )
        df = pd.concat([df, df_status], ignore_index=True, sort=True)
        return df

In [9]:
%%time
ObjectiveFunc.adset_status(23843355587430564)

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 438 ms


,adset_id,bid_amount,budget_per_day,campaign_id,charge,charge_cpc,charge_per_day,daily_budget,impressions,spend
0,2.384336e+16,16.0,1666.67,2.384336e+16,121.0,7.3719,333.333333,10000.0,5110.0,892.0


In [ ]:
%%time
camp_dict = mysql_adactivity_save.get_campaign_target_dict()

for campaign_id in camp_dict.keys():

    mydb = mysql_adactivity_save.connectDB( "ad_activity" )
    df_score = pd.read_sql("SELECT * FROM optimal_weight WHERE campaign_id=%s " %(campaign_id), con=mydb)
    ad_id_list = Campaigns(campaign_id).get_adids()
    for ad_id in ad_id_list:
        df = ObjectiveFunc.adset_status(ad_id)
        r = ObjectiveFunc.adset_fitness( optimal_weight, df )
        df_ad=pd.read_sql("SELECT adset_id FROM ad_insights WHERE ad_id=%s LIMIT 1" %(ad_id), con=mydb)
        adset_id = df_ad['adset_id'].iloc[0].astype(dtype=object)  
        df_final = pd.DataFrame({'campaign_id':camp_id, 'adset_id':adset_id, 'ad_id':ad_id, 'score':r}, index=[0])
        print(df_final)
        print(df[['charge', 'charge_cpc', 'impressions', 'spend', 'budget_per_day', 'bid_amount']])

In [ ]:
import math

#目標函數
def fitnessfunc(vardim, optimal_weight, bound, adset_id):
    w_kpi   = x[0]
    w_spend = x[1]
    w_bid   = x[2]
    w_ctr   = x[3]
    w_width = x[4]
    
    daily_clicks = 36
    budget_t     = 185
    bid          = 11
    target_ctr   = 2
    budget       = 1000
    
    clicks       = 53
    spend        = 230
    cpc          = 10
    ctr          = 1.74
    impressions  = 3040
    r = np.dot( optimal_weight, status )
    r = w_kpi * (clicks) / daily_clicks + w_spend * (budget_t-spend) / budget_t + w_bid * max( (bid-cpc) / bid , 0 )+ w_ctr * ctr / target_ctr + w_width*impressions/budget
    return r
def GrieFunc(vardim, x, bound):
    """
    Griewangk function
    """
    s1 = 0.
    s2 = 1.
    for i in range(1, vardim + 1):
        s1 = s1 + x[i - 1] ** 2
        s2 = s2 * math.cos(x[i - 1] / math.sqrt(i))
    y = (1. / 4000.) * s1 - s2 + 1
    y = 1. / (1. + y)
    return y

#非凸優化函數
def RastFunc(vardim, x, bound):
    """
    Rastrigin function
    """
    s = 10 * 25
    for i in range(1, vardim + 1):
        s = s + x[i - 1] ** 2 - 10 * math.cos(2 * math.pi * x[i - 1])
    return s

Campaign Assessment

In [ ]:
clicks = 46
spend  = 232
cpc    = 8
ctr    = 1.28
budget = 1000
impressions = 2195
status = [ clicks, spend, cpc, ctr, budget, impressions ]

In [ ]:
cal_fitness(status)

Adset Assessment

In [ ]:
STATUS = [ [ 102, 499, 9, 1.78, 1000, 3138 ],
           [ 118, 533, 9, 1.11, 1000, 5423 ],
           [ 77, 410, 9, 1.34, 1000, 3497 ],
           [ 120, 495, 9, 0.86, 1000, 6257 ],
           [ 56, 199, 9, 1.17, 1000, 1876 ],
           [ 12, 71, 10, 0.65, 1000, 1083 ],
           [ 87, 460, 9, 1.09, 1000, 4868 ],
           [ 99, 462, 9, 0.99, 1000, 5235 ],
           [ 22, 95, 10, 3.04, 1000, 329 ],]
len(STATUS)

In [ ]:
for i in range(0,len(STATUS)):
    print(cal_fitness(STATUS[i]))

4 2 8 7 1 3 5 9 6

In [ ]:
STATUS = [ [ 27, 166, 10, 1.18, 1000, 1436 ],
           [ 28, 173, 10, 1.79, 1000, 1007 ],
           [ 32, 190, 10, 1.49, 1000, 1275 ],
           [ 18, 132, 10, 2.21, 1000, 588 ],
           [ 16, 73, 10, 1.55, 1000, 451 ],
           [ 2, 21, 11, 1.01, 1000, 198 ],
           [ 24, 166, 10, 1.93, 1000, 883 ],
           [ 38, 191, 10, 0.88, 1000, 2260 ],
           [ 5, 29, 10, 3.66, 1000, 82 ],]

In [ ]:
for i in range(0,len(STATUS)):
    print(cal_fitness(STATUS[i]))

8 3 1 2 7 4 9 5 6

Objective Assessment - Post Engagement

In [ ]:
status=[ 341, 1356, 4, 15.34, 1650, 2231 ]
cal_fitness(status)

In [ ]:
import datetime
datetime.datetime.now().date()